<a href="https://colab.research.google.com/github/JacopoMangiavacchi/Swift-TensorFlow-Sample-Notebooks/blob/master/LinearRegression_MultiVariable_SciKit_Normalize_Swift_TensorFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import Python

func getNumpyNormalizedDataset() -> (PythonObject, PythonObject) {
    let np = Python.import("numpy")
    let pd = Python.import("pandas")
    let io = Python.import("io")
    let requests = Python.import("requests")
    let preprocessing = Python.import("sklearn.preprocessing")


    let url="https://raw.githubusercontent.com/Dataweekends/zero_to_deep_learning_video/master/data/weight-height.csv"
    let s = requests.get(url).content
    let df = pd.read_csv(io.StringIO(s.decode("utf-8")))

    let dummies = pd.get_dummies(df[["Gender"]])
    let trasnformed = pd.concat([df[["Height", "Weight"]], dummies], 1)
    // print(trasnformed)

    let X = trasnformed[["Height","Gender_Female","Gender_Male"]].values
    let Y = trasnformed[["Weight"]].values

    let scaler = preprocessing.MinMaxScaler()
    let xNP = np.array(scaler.fit_transform(X))
    let yNP = np.array(scaler.fit_transform(Y))  
    
    return (xNP, yNP)
}

In [0]:
import TensorFlow

In [0]:
@differentiable()
func linear(input: Tensor<Float>) -> Tensor<Float> {
    return input
}

struct LinearRegression: Layer {
    var l1: Dense<Float>
    init(variables: Int = 1) {
        l1 = Dense<Float>(inputSize: variables, outputSize: 1, activation: linear)
    }
    
    @differentiable(wrt: (self, input))
    func applied(to input: Tensor<Float>) -> Tensor<Float> {
        return l1.applied(to: input)
    }
}

In [4]:
let (xNP, yNP) = getNumpyNormalizedDataset()
// print(xNP.shape)
// print(yNP.shape)
// print(Python.isinstance(xNP,(np.ndarray)))

// let x = Tensor<Float>(numpy: xNP)
// let y = Tensor<Float>(numpy: yNP)


let xArray = xNP.tolist().flatMap{ $0.map{ Float($0)! }}
let yArray = yNP.tolist().flatMap{ $0.map{ Float($0)! }}

let x = Tensor<Float>(shape: [Int32(10000), 3], scalars: xArray)
let y = Tensor<Float>(shape: [Int32(10000), 1], scalars: yArray)

print(x.shape)
print(y.shape)

TensorShape(dimensions: [10000, 3])
TensorShape(dimensions: [10000, 1])


In [5]:
let optimizer = SGD<LinearRegression, Float>(learningRate: 0.03)
var model = LinearRegression(variables: 3)


for epoch in 1...2000 {
    let (cost, 𝛁model) = model.valueWithGradient { m -> Tensor<Float> in
        let ŷ = m.applied(to: x)
        return meanSquaredError(predicted: ŷ, expected: y)
    }
    optimizer.update(&model.allDifferentiableVariables, along: 𝛁model)
  
    if epoch % 100 == 0 {
        print("Epoch: \(epoch) Cost: \(cost)")
    }
}

Epoch: 100 Cost: 0.02094068
Epoch: 200 Cost: 0.016899573
Epoch: 300 Cost: 0.015150159
Epoch: 400 Cost: 0.013614702
Epoch: 500 Cost: 0.012263853
Epoch: 600 Cost: 0.011075411
Epoch: 700 Cost: 0.0100298505
Epoch: 800 Cost: 0.009109996
Epoch: 900 Cost: 0.008300731
Epoch: 1000 Cost: 0.0075887595
Epoch: 1100 Cost: 0.0069623864
Epoch: 1200 Cost: 0.0064113215
Epoch: 1300 Cost: 0.0059265085
Epoch: 1400 Cost: 0.0054999804
Epoch: 1500 Cost: 0.0051247347
Epoch: 1600 Cost: 0.0047946023
Epoch: 1700 Cost: 0.004504159
Epoch: 1800 Cost: 0.004248636
Epoch: 1900 Cost: 0.004023834
Epoch: 2000 Cost: 0.0038260582


In [6]:
print(model.applied(to: [[0.7, 0, 1]]))  //Height, Female, Male

[[0.63725114]]
